In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.control import MPCHybridController
from pympc.feasible_set_library import FeasibleSetLibrary, load_library
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt

# Construction of Box-Atlas

Robot moving limbs

In [ ]:
from pympc.models.boxatlas_parameters import dynamics
penetration = dynamics['mass'] * dynamics['gravity'] / dynamics['normal_stiffness']

# right hand
A_rh = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[1., 0.],[-1., 0.],[0., -1.]]),
}
b_rh = {
    'not_in_contact': np.array([[.4],[.5]]),
    'in_contact': np.array([[-.4],[.4+penetration],[.5]]),
}
contact_surfaces_rh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rh = np.array([[-.3], [.0]])
parametric_wall_distance = {
    'label': 'wall_distance_rh',
    'not_in_contact': {'index': [0], 'coefficient': [1.]},
    'in_contact': {'index': [0, 1], 'coefficient': [-1., 1.]},
    'min': -.05,
    'max': .05,
} # if positive moves the wall away from the body
parameters_rh = [parametric_wall_distance]
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh, parameters=parameters_rh)

# left foot
A_lf = {
    'not_in_contact': np.array([[0., -1.]]),
    'in_contact': np.array([[0., 1.],[0., -1.]]),
}
b_lf = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+2.*penetration]]),
}
contact_surfaces_lf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lf = np.array([[.2], [-.5 - penetration / 2.]])
lf = MovingLimb(A_lf, b_lf, contact_surfaces_lf, q_lf)

# right foot
A_rf = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[0., 1.],[-1., 0.],[0., -1.]]),
}
b_rf = {
    'not_in_contact': np.array([[.5],[.5]]),
    'in_contact': np.array([[-.5],[.5],[.5+2.*penetration]]),
}
contact_surfaces_rf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rf = np.array([[-.2], [-.5 - penetration / 2.]])
rf = MovingLimb(A_rf, b_rf, contact_surfaces_rf, q_rf)

Assemble robot

In [ ]:
limbs = { 'moving': {'rh': rh, 'lf': lf, 'rf': rf}, 'fixed': {} }
nominal_mode = {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
box_atlas = BoxAtlas(limbs, nominal_mode)

Print state, input, and modes

In [ ]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
box_atlas.print_modes()

# Hybrid MPC

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = .0
x_0[10,0] = -1.5
u, x, ss = box_atlas.controller.feedforward(x_0)[0:3]

Closed-loop system simulation of the MPC controller

In [ ]:
N_sim = 30
u = []
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = .05
x_0[10,0] = -1.5
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
nominal_mode = box_atlas.get_mode_index(box_atlas.nominal_mode)
for k in range(N_sim):
    u_k, x_k, ss_k = box_atlas.controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print 'Time step ' + str(k) + ': ' + str(ss_k) + '.'
    u_ws = u_k[1:] + [box_atlas.K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (nominal_mode,)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    # print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)
# traj_miqp = Trajectory(x, u, box_atlas.Q, box_atlas.R, box_atlas.P)
# np.save('box_atlas_tajectory_miqp_lh', traj_miqp)

Visualization of the simulation with Director

In [ ]:
from pympc.models.boxatlas_parameters import dynamics
box_atlas.visualize(x[0])
time.sleep(10 * dynamics['sampling_time'])
for x_t in x:
    box_atlas.visualize(x_t)
    time.sleep(dynamics['sampling_time'])

# Approximate Hybrid MPC

Initialization of the library

In [ ]:
library = FeasibleSetLibrary(box_atlas.controller)
samples_so_far = 0
# library = load_library('library_box_atlas_' + str(samples_so_far))

Coverage

In [ ]:
sample_step = 100
n_steps = 100
for i in range(n_steps):
    n = samples_so_far + (i+1)*sample_step
    library.sample_policy(sample_step, box_atlas.is_inside_a_domain)
    print(str(n) + ' samples added to the library.')
    library.save('library_box_atlas_moving_feet_' + str(n))

Shifted switching sequences

In [ ]:
library.add_shifted_switching_sequences(nominal_mode)

Closed-loop system simulation of the approximate MPC controller

In [ ]:
u = []
x = [x_0]
ss = []
ss_feasible = None
for k in range(N_sim):
    print 'Time step ' + str(k) + '.'
    u_k, ss_k = library.feedback(x[k], ss_feasible, max_qp=10)
    u.append(u_k)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k])[0][1]
    x.append(x_next)
    ss.append(ss_k)
    ss_feasible = ss_k[1:] + (nominal_mode,)
traj_lib = Trajectory(x, u, box_atlas.Q, box_atlas.R, box_atlas.P)
#np.save('box_atlas_tajectory', traj_lib)